In [1]:
import pandas as pd
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
pd.set_option('display.max_columns', None)

In [517]:
df = pd.read_csv('train.csv')
df

,Unnamed: 0,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus
0,0,Slightly disagree,Neutral,Slightly agree,Slightly agree,Slightly agree,Neutral,Slightly agree,Neutral,Neutral,Neutral,Disagree,Slightly disagree,Neutral,Disagree,Neutral,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75
1,1,Slightly agree,Slightly agree,Slightly disagree,Slightly agree,Disagree,Slightly disagree,Slightly disagree,Slightly agree,Slightly agree,Agree,Slightly disagree,Slightly agree,Slightly agree,Slightly disagree,Neutral,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00
2,2,Disagree,Slightly disagree,Agree,Slightly agree,Slightly agree,Slightly agree,Slightly disagree,Slightly agree,Slightly agree,Slightly agree,Slightly disagree,Slightly disagree,Slightly agree,Disagree,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00
3,3,Neutral,Slightly agree,Agree,Slightly agree,Slightly agree,Slightly agree,Agree,Neutral,Slightly disagree,Slightly agree,Neutral,Neutral,Slightly agree,Slightly disagree,Slightly disagree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00
4,4,Neutral,Slightly disagree,Neutral,Slightly disagree,Agree,Slightly agree,Slightly agree,Disagree,Slightly agree,Agree,Slightly disagree,Slightly agree,Slightly agree,Neutral,Neutral,Slightly agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,12995,Neutral,Slightly agree,Agree,Slightly agree,Slightly agree,Neutral,Slightly agree,Neutral,Slightly agree,Slightly agree,Slightly disagree,Neutral,Slightly agree,Slightly disagree,Slightly disagree,Neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00
12996,12996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agree,Agree,Disagree,Slightly agree,Slightly disagree,Agree,Slightly disagree,Disagree,Agree,Agree,Neutral,Slightly disagree,Agree,Slightly agree,Disagree,Neutral,2.00
12997,12997,Slightly disagree,Neutral,Agree,Slightly agree,Slightly agree,Slightly disagree,Slightly agree,Neutral,Neutral,Neutral,Slightly agree,Agree,Agree,Neutral,Slightly agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50
12998,12998,Neutral,Disagree,Neutral,Neutral,Disagree,Neutral,Neutral,Disagree,Neutral,Neutral,Disagree,Neutral,Disagree,Disagree,Slightly agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00


Transform categorical features to numbers:

In [518]:
transformation = {'Disagree': 1, 'Slightly disagree': 2, 'Neutral': 3,
                  'Slightly agree': 4, 'Agree': 5}

In [519]:
df = df.drop(columns=['Unnamed: 0'])

In [520]:
df = df.replace(transformation)

## Restoring uids

In [388]:
print('Probability of coincidence of answers of 2 different users: ', 1000000 / 5 ** 16)

Probability of coincidence of answers of 2 different users:  6.5536e-06


##### Assumption:
Users with the same answers are the same user

In [521]:
column_name = 'uid_question_'
uids, match_uids = [], []


for i, row in df.iterrows():
    uid, match_uid = '', ''
    for j in range(16):
        if pd.isna(row[column_name + str(j)]):
            uid = np.nan
        else:
            uid += str(row[column_name + str(j)])

        if pd.isna(row['match_' + column_name + str(j)]):
            match_uid = np.nan
        else:
            match_uid += str(row['match_' + column_name + str(j)])
    uids.append(uid)
    match_uids.append(match_uid)
            
            

In [522]:
hash_to_uid = dict()

def create_uids_matching(uids, i):
    for el in set(uids):
        if el is np.nan:
            continue
        if el not in hash_to_uid:
            hash_to_uid[el] = i
            i += 1
    return i

i = create_uids_matching(uids, 0)
create_uids_matching(match_uids, i)

335

In [523]:
final_uid, final_match_uid = [], []

j = 335
for uid, match_uid in zip(uids, match_uids):
    if uid is np.nan:
        final_uid.append(j)
        j += 1
    else:
        final_uid.append(hash_to_uid[uid])
    if match_uid is np.nan:
        final_match_uid.append(j)
        j += 1
    else:
        final_match_uid.append(hash_to_uid[match_uid])
        

In [524]:
# sanity check
len(final_uid), len(final_match_uid)

(13000, 13000)

In [525]:
df['uid'] = final_uid
df['match_uid'] = final_match_uid

In [526]:
df

,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus,uid,match_uid
0,2.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0,1.0,2.0,3.0,1.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,335,185
1,4.0,4.0,2.0,4.0,1.0,2.0,2.0,4.0,4.0,5.0,2.0,4.0,4.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,336,220
2,1.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,4.0,1.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,337,119
3,3.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,2.0,4.0,3.0,3.0,4.0,2.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,338,125
4,3.0,2.0,3.0,2.0,5.0,4.0,4.0,1.0,4.0,5.0,2.0,4.0,4.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,339,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,3.0,4.0,5.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,2.0,3.0,4.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,12925,25
12996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,1.0,4.0,2.0,5.0,2.0,1.0,5.0,5.0,3.0,2.0,5.0,4.0,1.0,3.0,2.00,320,12926
12997,2.0,3.0,5.0,4.0,4.0,2.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.50,12927,296
12998,3.0,1.0,3.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,1.0,3.0,1.0,1.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,12928,123


Calculating mean and std of scores that users give and get

In [532]:
def calc_loo_mean(df, by):
    gb = df.groupby(by).star_plus.agg(['sum', 'count'])
    gb['count'] -= 1
    gb = pd.merge(df, gb, how='left', on=by)
    gb['sum'] -= gb.star_plus
    gb['mean'] = gb['sum'] / gb['count']
    return gb.drop(columns=['sum', 'count']).rename(columns={'mean': by + '_mean'})


def calc_loo_std(df, by):
    gb = df.groupby(by).star_plus.agg(['count', 'std'])
    gb['std'] *= gb['count']
    gb = pd.merge(df, gb, how='left', on=by)
    gb['std'] -= (gb['star_plus'] - gb[by + '_mean']) ** 2
    gb['count'] -= 1
    gb['std'] /= gb['count']
    return gb.drop(columns=['count']).rename(columns={'std': by + '_std'})

In [534]:
df = calc_loo_std(df, 'uid')
df = calc_loo_std(df, 'match_uid')
df.head()

,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus,uid,match_uid,uid_mean,match_uid_mean,uid_std,match_uid_std
0,2.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0,1.0,2.0,3.0,1.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,335,185,NaN,2.903226,NaN,1.220087
1,4.0,4.0,2.0,4.0,1.0,2.0,2.0,4.0,4.0,5.0,2.0,4.0,4.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,336,220,NaN,3.791667,NaN,1.259172
2,1.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,4.0,1.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,337,119,NaN,3.636574,NaN,1.397126
3,3.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,2.0,4.0,3.0,3.0,4.0,2.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,338,125,NaN,3.701220,NaN,1.277582
4,3.0,2.0,3.0,2.0,5.0,4.0,4.0,1.0,4.0,5.0,2.0,4.0,4.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,339,5,NaN,4.205357,NaN,1.191795


## Model for samples where both users have features

In [535]:
full_info = df.loc[(df['uid'] < 335) & (df['match_uid'] < 335)]

In [536]:
full_info.head()

,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus,uid,match_uid,uid_mean,match_uid_mean,uid_std,match_uid_std
12,2.0,2.0,1.0,2.0,4.0,1.0,3.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,5.0,3.0,4.0,4.0,4.0,2.0,2.0,4.0,3.0,1.0,4.0,4.0,1.0,4.0,2.0,2.0,5.0,2.0,78,303,4.789062,3.806373,1.050567,1.263853
79,1.0,4.0,5.0,4.0,2.0,2.0,4.0,2.0,1.0,3.0,4.0,4.0,4.0,3.0,2.0,5.0,4.0,4.0,1.0,3.0,2.0,3.0,1.0,1.0,5.0,5.0,1.0,5.0,3.0,4.0,3.0,1.0,5.0,71,169,3.758333,3.833333,1.544458,1.625480
119,1.0,5.0,5.0,5.0,5.0,3.0,5.0,1.0,3.0,5.0,1.0,5.0,5.0,5.0,1.0,5.0,5.0,4.0,5.0,4.0,2.0,5.0,2.0,2.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,5.0,0.5,205,68,3.122340,2.725000,0.937862,1.546077
131,4.0,5.0,1.0,5.0,4.0,2.0,1.0,1.0,3.0,5.0,3.0,3.0,5.0,3.0,3.0,5.0,4.0,5.0,1.0,3.0,5.0,4.0,1.0,3.0,5.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,203,115,4.138889,3.418182,1.243525,1.224402
134,4.0,1.0,4.0,5.0,2.0,5.0,1.0,4.0,1.0,1.0,2.0,4.0,4.0,3.0,2.0,4.0,3.0,3.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,2.0,4.0,5.0,2.0,1.0,4.0,5.0,174,108,3.592105,3.336538,1.136695,1.151609


Calculate differences btwn answers:

In [537]:
new_columns = dict()

for i in range(16):
    new_columns['question_' + str(i)] = full_info['uid_question_' + str(i)] - full_info['match_uid_question_' + str(i)]

In [538]:
full_info_train = pd.DataFrame(new_columns)#.reset_index().drop('index', axis=1)

In [539]:
full_info_train.head()

,question_0,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10,question_11,question_12,question_13,question_14,question_15
12,1.0,2.0,0.0,3.0,3.0,0.0,1.0,1.0,1.0,0.0,3.0,2.0,-2.0,1.0,1.0,-1.0
79,3.0,0.0,4.0,2.0,-3.0,1.0,-1.0,1.0,1.0,-4.0,-4.0,-1.0,0.0,1.0,-3.0,-1.0
119,4.0,-1.0,1.0,-1.0,3.0,-1.0,0.0,-1.0,2.0,0.0,0.0,-1.0,-3.0,2.0,-3.0,1.0
131,0.0,0.0,2.0,-1.0,-2.0,-2.0,-3.0,0.0,-2.0,-2.0,0.0,-2.0,1.0,2.0,0.0,2.0
134,-1.0,2.0,3.0,4.0,0.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,2.0,-1.0,3.0,0.0


In [540]:
new_column_sum = full_info_train.sum(axis=1)
new_column_std = full_info_train.std(axis=1)

In [541]:
full_info['sum'] = new_column_sum
full_info['std'] = new_column_std
full_info.head()

/home/ivan/venvs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ivan/venvs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus,uid,match_uid,uid_mean,match_uid_mean,uid_std,match_uid_std,sum,std
12,2.0,2.0,1.0,2.0,4.0,1.0,3.0,4.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,5.0,3.0,4.0,4.0,4.0,2.0,2.0,4.0,3.0,1.0,4.0,4.0,1.0,4.0,2.0,2.0,5.0,2.0,78,303,4.789062,3.806373,1.050567,1.263853,16.0,1.414214
79,1.0,4.0,5.0,4.0,2.0,2.0,4.0,2.0,1.0,3.0,4.0,4.0,4.0,3.0,2.0,5.0,4.0,4.0,1.0,3.0,2.0,3.0,1.0,1.0,5.0,5.0,1.0,5.0,3.0,4.0,3.0,1.0,5.0,71,169,3.758333,3.833333,1.544458,1.625480,-4.0,2.380476
119,1.0,5.0,5.0,5.0,5.0,3.0,5.0,1.0,3.0,5.0,1.0,5.0,5.0,5.0,1.0,5.0,5.0,4.0,5.0,4.0,2.0,5.0,2.0,2.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,5.0,0.5,205,68,3.122340,2.725000,0.937862,1.546077,2.0,1.962142
131,4.0,5.0,1.0,5.0,4.0,2.0,1.0,1.0,3.0,5.0,3.0,3.0,5.0,3.0,3.0,5.0,4.0,5.0,1.0,3.0,5.0,4.0,1.0,3.0,5.0,4.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,203,115,4.138889,3.418182,1.243525,1.224402,-7.0,1.631717
134,4.0,1.0,4.0,5.0,2.0,5.0,1.0,4.0,1.0,1.0,2.0,4.0,4.0,3.0,2.0,4.0,3.0,3.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,2.0,4.0,5.0,2.0,1.0,4.0,5.0,174,108,3.592105,3.336538,1.136695,1.151609,11.0,1.621471


In [587]:
train = full_info[['uid_mean', 'match_uid_mean', 'sum', 'std', 'uid_std', 'match_uid_std' ]] #'uid_std', 'match_uid_std' , 'sum', 'std' 
train.fillna(4., inplace=True) # good for mean and std
train.head()

/home/ivan/venvs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:4465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,uid_mean,match_uid_mean,sum,std,uid_std,match_uid_std
12,4.789062,3.806373,16.0,1.414214,1.050567,1.263853
79,3.758333,3.833333,-4.0,2.380476,1.544458,1.625480
119,3.122340,2.725000,2.0,1.962142,0.937862,1.546077
131,4.138889,3.418182,-7.0,1.631717,1.243525,1.224402
134,3.592105,3.336538,11.0,1.621471,1.136695,1.151609


In [588]:
from sklearn.model_selection import train_test_split


X, y = train, full_info['star_plus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [589]:
X_train

,uid_mean,match_uid_mean,sum,std,uid_std,match_uid_std
11337,3.510417,3.864583,-7.0,1.412740,1.243411,1.308348
7537,4.330729,3.857143,-19.0,2.007278,1.045463,1.437743
131,4.138889,3.418182,-7.0,1.631717,1.243525,1.224402
545,4.125000,3.504386,1.0,2.205108,1.139698,1.308592
4228,3.638889,2.532609,4.0,1.949359,1.301825,0.943638
...,...,...,...,...,...,...
2361,4.665094,3.700000,4.0,1.914854,0.824659,1.107748
3509,2.226562,2.000000,10.0,1.258306,1.406656,-0.128680
8660,4.343254,3.584821,-3.0,2.040221,1.051972,1.376143
11108,3.677083,3.614754,22.0,1.586401,1.106027,1.262569


In [590]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import r2_score


regr = RFR(max_depth=1, random_state=42, n_estimators=100)
regr.fit(X_train, y_train)
round(r2_score(y_test, regr.predict(X_test)), 6), round(mse(y_test, regr.predict(X_test), squared=False), 6)

(0.033125, 1.214896)

In [591]:
from sklearn.linear_model import Ridge

lr = Ridge(alpha=1)
lr.fit(X_train, y_train)
round(r2_score(y_test, lr.predict(X_test)), 6)

0.04248

In [593]:
def highlight_max(x):
    return ['font-weight: bold' if v == x.loc[4] else ''
                for v in x]

In [596]:
results = pd.DataFrame({'exp': ['basic', 'sum&std', 'mean uid&match_uid', 'mean&std uid&match_uid', 'mean uid&match_uid & sum&std', 'all_feats'],
                        'Random Forest': [0.007941, 0.027129, 0.026905, 0.027759, 0.033644, 0.033125],
                        'LinReg': [-0.011097, 0.020192, 0.028598, 0.026776, 0.047475, 0.04248]})
results.style.apply(highlight_max)

,exp,Random Forest,LinReg
0,basic,0.007941,-0.011097
1,sum&std,0.027129,0.020192
2,mean uid&match_uid,0.026905,0.028598
3,mean&std uid&match_uid,0.027759,0.026776
4,mean uid&match_uid & sum&std,0.033644,0.047475
5,all_feats,0.033125,0.042480


In [655]:
0.047 / 0.028

1.6785714285714286

In [604]:
print('Increased quality from baseline in ', round((0.047475 - (-0.011097)) / 0.011097, 1), 'times')

Increased quality from baseline in  5.3 times


In [651]:
# base = 0.007941 (-0.011097 lr)
# sum_std = 0.027129 (0.020192 lr)
# mean_uid_match_uid = 0.026905 (0.028598 lr)
# mean_std_uid_match_uid = 0.027759 (0.026776 lr)
# mean_uid_match_uid_sum_std = 0.033644 (0.047475 lr)
# all_feats = 0.033125 (0.04248 lr)

##### Next steps: 

1. Use different scale of transformation (for example 1 3 4 5 7 instead of 1 2 3 4 5) to highlite bigger difference between peak answers

2. predict random forest and use it as a feature in lr

3. use GBDT on bigger dataset

4. use number of scores as param

5. Collect more data

## Model for samples with only one set of features

##### Problem:
Features do not make sense because we don't know how they relate with other user's features

##### Solution:
Make prediction based on mean score user gives or gets

In [605]:
partial_info = df.loc[(df['uid'] >= 335) | (df['match_uid'] >= 335)]

In [606]:
partial_info.head()

,match_uid_question_0,match_uid_question_1,match_uid_question_10,match_uid_question_11,match_uid_question_12,match_uid_question_13,match_uid_question_14,match_uid_question_15,match_uid_question_2,match_uid_question_3,match_uid_question_4,match_uid_question_5,match_uid_question_6,match_uid_question_7,match_uid_question_8,match_uid_question_9,uid_question_0,uid_question_1,uid_question_10,uid_question_11,uid_question_12,uid_question_13,uid_question_14,uid_question_15,uid_question_2,uid_question_3,uid_question_4,uid_question_5,uid_question_6,uid_question_7,uid_question_8,uid_question_9,star_plus,uid,match_uid,uid_mean,match_uid_mean,uid_std,match_uid_std
0,2.0,3.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,3.0,1.0,2.0,3.0,1.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,335,185,NaN,2.903226,NaN,1.220087
1,4.0,4.0,2.0,4.0,1.0,2.0,2.0,4.0,4.0,5.0,2.0,4.0,4.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,336,220,NaN,3.791667,NaN,1.259172
2,1.0,2.0,5.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,2.0,2.0,4.0,1.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,337,119,NaN,3.636574,NaN,1.397126
3,3.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,2.0,4.0,3.0,3.0,4.0,2.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.00,338,125,NaN,3.701220,NaN,1.277582
4,3.0,2.0,3.0,2.0,5.0,4.0,4.0,1.0,4.0,5.0,2.0,4.0,4.0,3.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,339,5,NaN,4.205357,NaN,1.191795


Drop users without any history:

In [613]:
absent_history = partial_info.isin({'uid_mean': [np.nan], 'match_uid_mean': [np.nan]})
absent_history = absent_history.loc[(absent_history['uid_mean'] == True) & (absent_history['match_uid_mean'] == True)]
indcs_to_drop = absent_history.index

In [615]:
partial_info.drop(index=indcs_to_drop, inplace=True)

/home/ivan/venvs/kaggle/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [650]:
# sanity check
len(partial_info)

12558

We will use uid_mean as predictor when match_uid_mean is null and vice versa

In [648]:
preds = []

for i, row in partial_info.iterrows():
    if pd.isna(row['uid_mean']):
        preds.append(row['match_uid_mean'])
    else:
        preds.append(row['uid_mean'])

In [649]:
print('R2 score: ', round(r2_score(partial_info['star_plus'], preds), 6))

R2 score:  0.05779


#### Next steps

We can try to model dependence between features and scores that user gives or gets